In [1]:
import json
import pprint
import pyspark as ps
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 200

In [3]:
spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("df case study") \
            .getOrCreate()

tweets_df = spark.read.json('./data/french_tweets.json').sample(False, 0.2)

In [5]:
tweets_df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [6]:
print(json.dumps(json.loads(pd_df.iloc[2].to_json()), indent=4)) 

{
    "contributors": null,
    "coordinates": null,
    "created_at": "Wed Apr 26 13:31:04 +0000 2017",
    "display_text_range": [
        0,
        140
    ],
    "entities": [
        [
            [
                [
                    57,
                    60
                ],
                "FN"
            ]
        ],
        null,
        [],
        [
            [
                "twitter.com/i/web/status/8\u2026",
                "https://twitter.com/i/web/status/857225518831325184",
                [
                    116,
                    139
                ],
                "https://t.co/w7p82ZG4Od"
            ]
        ],
        [
            [
                1976143068,
                "1976143068",
                [
                    82,
                    97
                ],
                "Emmanuel Macron",
                "EmmanuelMacron"
            ]
        ]
    ],
    "extended_entities": null,
    "extended_tweet": [
        [
         

In [7]:
tweets_df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

In [8]:
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43429 entries, 0 to 43428
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   contributors               0 non-null      object 
 1   coordinates                3628 non-null   object 
 2   created_at                 43427 non-null  object 
 3   display_text_range         21422 non-null  object 
 4   entities                   43427 non-null  object 
 5   extended_entities          4503 non-null   object 
 6   extended_tweet             4513 non-null   object 
 7   favorite_count             43427 non-null  float64
 8   favorited                  43427 non-null  object 
 9   filter_level               43427 non-null  object 
 10  geo                        3628 non-null   object 
 11  id                         43427 non-null  float64
 12  id_str                     43427 non-null  object 
 13  in_reply_to_screen_name    12736 non-null  obj

In [93]:
def coords_estimate(entry):
    '''
    params: row: the individual row of the dataframe
    
    returns: an array of length 2 with the average x and y coordinates of the bounding box.
    
    '''
    coords = np.array(entry[0])
    return np.mean(coords, axis = 0) 

pd_df = cleaned.toPandas()
pd_df['coord_estimates'] = pd_df['place_coordinates'].dropna().apply(coords_estimate)

In [53]:
def CleanUp(rdd, columns):
    return (rdd[columns].withColumn('user_id', rdd['user.id'])
            .withColumn('user_screen_name', rdd['user.screen_name'])
            .withColumn('place_type', rdd['place.place_type'])
            .withColumn('place_coordinates', rdd['place.bounding_box.coordinates'])
            .withColumn('hashtags', rdd['entities.hashtags.text'])
            .withColumn('mentions_id', rdd['entities.user_mentions.id'])
            .drop('user')
            .drop('place')
            .drop('entities')
           )
cleaned = CleanUp(tweets_df, ['entities', 'filter_level', 'id', 'in_reply_to_user_id', 'lang', 'place', 'possibly_sensitive', 'text', 'timestamp_ms', 'user'])

In [94]:
coords_estimate(pd_df['place_coordinates'].iloc[3])

array([ 3.8742675, 43.610396 ])

In [95]:
pd_df['coord_estimates'] = pd_df['place_coordinates'].dropna().apply(coords_estimate)
pd_df['coord_estimates'].head()

0                    [-1.6197325, 47.274349]
1                     [2.347003, 48.8588335]
2             [2.568002, 49.028848499999995]
3                     [3.8742675, 43.610396]
4    [6.143354500000001, 49.123067999999996]
Name: coord_estimates, dtype: object

In [12]:
cleaned.createOrReplaceTempView('df') 

In [44]:
result = spark.sql('''
        SELECT place_coordinates
        FROM df 
''')

Column<'place_coordinates'>

In [34]:
result = spark.sql('''
        SELECT place_type, count(place_type)/ (SELECT count(*) FROM df ) proportion
        FROM df
        GROUP BY place_type
''').show()
result

+----------+-------------------+
|place_type|         proportion|
+----------+-------------------+
|      null|                0.0|
|      city|  0.830343779502176|
|   country|0.05652904741071634|
|     admin| 0.1119988947477492|
+----------+-------------------+



In [35]:
result_m = spark.sql('''
        WITH df_m as (SELECT * FROM df WHERE support = 1) 
        SELECT place_type, count(place_type)/ (SELECT count(*) FROM df_m ) proportion_m
        FROM df_m
        GROUP BY place_type
''').show()
result_m

AnalysisException: cannot resolve '`support`' given input columns: [df.filter_level, df.hashtags, df.id, df.in_reply_to_user_id, df.lang, df.mentions_id, df.place_coordinates, df.place_type, df.possibly_sensitive, df.text, df.timestamp_ms, df.user_id, df.user_screen_name]; line 2 pos 45;
'Aggregate ['place_type], ['place_type, ('count('place_type) / scalar-subquery#518 []) AS proportion_m#519]
:  +- 'Project [unresolvedalias('count(1), None)]
:     +- 'SubqueryAlias df_m
:        +- 'Project [*]
:           +- 'Filter ('support = 1)
:              +- 'UnresolvedRelation [df], [], false
+- 'SubqueryAlias df_m
   +- 'Project [*]
      +- 'Filter ('support = 1)
         +- SubqueryAlias df
            +- Project [filter_level#16, id#18L, in_reply_to_user_id#23L, lang#26, possibly_sensitive#29, text#36, timestamp_ms#37, user_id#117L, user_screen_name#130, place_type#144, place_coordinates#159, hashtags#175, mentions_id#192]
               +- Project [entities#11, filter_level#16, id#18L, i

In [33]:
result_p = spark.sql('''
        WITH df_p as (SELECT * FROM df WHERE support = 0)
        SELECT place_type, count(place_type)/ (SELECT count(*) FROM df_p ) proportion_p
        FROM df_p
        GROUP BY place_type
''').show()
result_m

AnalysisException: cannot resolve '`support`' given input columns: [df.filter_level, df.hashtags, df.id, df.in_reply_to_user_id, df.lang, df.mentions_id, df.place_coordinates, df.place_type, df.possibly_sensitive, df.text, df.timestamp_ms, df.user_id, df.user_screen_name]; line 2 pos 45;
'Aggregate ['place_type], ['place_type, ('count('place_type) / scalar-subquery#494 []) AS proportion_p#495]
:  +- 'Project [unresolvedalias('count(1), None)]
:     +- 'SubqueryAlias df_p
:        +- 'Project [*]
:           +- 'Filter ('support = 0)
:              +- 'UnresolvedRelation [df], [], false
+- 'SubqueryAlias df_p
   +- 'Project [*]
      +- 'Filter ('support = 0)
         +- SubqueryAlias df
            +- Project [filter_level#16, id#18L, in_reply_to_user_id#23L, lang#26, possibly_sensitive#29, text#36, timestamp_ms#37, user_id#117L, user_screen_name#130, place_type#144, place_coordinates#159, hashtags#175, mentions_id#192]
               +- Project [entities#11, filter_level#16, id#18L, i

In [29]:
result = spark.sql('''
        WITH total as  (SELECT count(*) FROM df )
        SELECT place_type, count(place_type)/
        FROM df
        GROUP BY place_type
''').show()
result

AnalysisException: cannot resolve '`total`' given input columns: [df.filter_level, df.hashtags, df.id, df.in_reply_to_user_id, df.lang, df.mentions_id, df.place_coordinates, df.place_type, df.possibly_sensitive, df.text, df.timestamp_ms, df.user_id, df.user_screen_name]; line 3 pos 46;
'Aggregate [place_type#144], [place_type#144, unresolvedalias((count(place_type#144) / 'total), None)]
+- SubqueryAlias df
   +- Project [filter_level#16, id#18L, in_reply_to_user_id#23L, lang#26, possibly_sensitive#29, text#36, timestamp_ms#37, user_id#117L, user_screen_name#130, place_type#144, place_coordinates#159, hashtags#175, mentions_id#192]
      +- Project [entities#11, filter_level#16, id#18L, in_reply_to_user_id#23L, lang#26, possibly_sensitive#29, text#36, timestamp_ms#37, user_id#117L, user_screen_name#130, place_type#144, place_coordinates#159, hashtags#175, mentions_id#192]
         +- Project [entities#11, filter_level#16, id#18L, in_reply_to_user_id#23L, lang#26, place#28, possibly_sens

In [64]:
coords_estimate(pd_df.iloc[3]), place_type(pd_df.iloc[])

(array([ 2.5696285, 44.3592395]), 'admin')